In [7]:
from tqdm.auto import tqdm
import torch 
import transformers
from transformers import PreTrainedTokenizer
from transformers import RobertaTokenizer, PreTrainedTokenizer, DistilBertTokenizer, DistilBertModel, RobertaModel
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, util
from datasets import Dataset
import pandas as pd
from transformers.optimization import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt 
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from torch.nn import CosineEmbeddingLoss
import random
from torch.nn import CosineEmbeddingLoss
from torch import nn
#Build up to SBERT model 

In [8]:
deviceNum = 3
device = torch.device("cuda:" + str(deviceNum) if torch.cuda.is_available() else "cpu")

GRAD_ACC = 6
EPOCHS = 2
FOLDS = 5
SEED = 85
BATCH_SIZE = 5

#set seeds 
torch.manual_seed(85)
random.seed(85)



In [9]:
#df = pd.read_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/translatedCleaned.tsv", sep="\t")
df = pd.read_csv("/home/blitt/projects/localNews/data/processed/translated_288_96.tsv", sep="\t")

#put ground truth values into a list 
df["ground_truth"] = df['Overall']

#get only the columns we need 
#TODO: do we need "pair_id"? 
#leanDf = df[["ground_truth",  'text1', 'text2', 'title1', 'title2', 'url1_lang', 'url2_lang']].dropna()
#for when using merged text
leanDf = df[["ground_truth",  'text1Merged', 'text2Merged', 'url1_lang', 'url2_lang']].dropna()

#rescale data from (0, 4): (0, 1)
leanDf["ground_truth"] = 1 - ((leanDf["ground_truth"] - 1) / 3)

#reset index so it is contiguous set of numbers 
leanDf = leanDf.reset_index(drop=True)



In [10]:
len(leanDf)

4806

In [6]:
1787 - 1738 

49

In [5]:
df[(df["url1_lang"] == "en") & (df["url2_lang"] == "en") ]

,Unnamed: 0,Unnamed: 0.1,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,...,title2,titleText1,titleText2,text1Head,text1Tail,text2Head,text2Tail,text1Merged,text2Merged,ground_truth
0,0,0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,...,Haiti’s leader marks independence day amid sec...,Virginia man arrested in fatal DUI crash in We...,Haiti’s leader marks independence day amid sec...,virginia man arrested in fatal dui crash in we...,released. the suspect ’ s vehicle also hit a d...,haiti ’ s leader marks independence day amid s...,been killed and dozens injured. large - scale ...,virginia man arrested in fatal dui crash in we...,haiti ’ s leader marks independence day amid s...,4.000000
1,1,1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,...,Fire kills more than 30 animals at zoo in west...,Guyana: Three injured after car crashes into u...,Fire kills more than 30 animals at zoo in west...,guyana : three injured after car crashes into ...,driver fled the scene and was subsequently app...,fire kills more than 30 animals at zoo in west...,", making them fly and shine at night. get more...",guyana : three injured after car crashes into ...,fire kills more than 30 animals at zoo in west...,3.666667
2,2,2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,...,"Trump says he does not expect war with Iran, ‘...",Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"Trump says he does not expect war with Iran, ‘...",trump brings in 2020 at mar - a - lago : ‘ we ...,so we don ’ t want to. i don ’ t want to say w...,"trump says he does not expect war with iran, ‘...","no plans to evacuate the mission, and no us pe...",trump brings in 2020 at mar - a - lago : ‘ we ...,"trump says he does not expect war with iran, ‘...",2.333333
3,3,3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber...,https://gadgets.ndtv.com/internet/news/indian-...,https://web.archive.org/web/gadgets.ndtv.com/a...,https://web.archive.org/web/gadgets.ndtv.com/i...,1.0,...,Indian Online Food Delivery Market to Hit $8 B...,Zomato Buys Uber's Food Delivery Business in I...,Indian Online Food Delivery Market to Hit $8 B...,zomato buys uber's food delivery business in i...,bangladesh and sri lanka. while this is the fi...,indian online food delivery market to hit $ 8 ...,"is the primary roadblock, "" the findings showe...",zomato buys uber's food delivery business in i...,indian online food delivery market to hit $ 8 ...,2.000000
4,4,4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-mo...,https://www.channelnewsasia.com/news/asia/indi...,https://web.archive.org/web/news.yahoo.com/ind...,https://web.archive.org/web/www.channelnewsasi...,1.0,...,India targets new moon mission in 2020,"India approves third moon mission, months afte...",India targets new moon mission in 2020BANGALOR...,"india approves third moon mission, months afte...","billion ), the government said in 2018. india ...",india targets new moon mission in 2020bangalor...,"$ 35 million, with a significantly higher outl...","india approves third moon mission, months afte...",india targets new moon mission in 2020bangalor...,1.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3319,3387,3413,en,en,1602628318_1522567203,https://communities.vmware.com/message/2948357...,https://communities.vmware.com/message/2927239...,https://web.archive.org/web/communities.vmware.

In [4]:
class BiModel(nn.Module): 
    def __init__(self):
        super(BiModel,self).__init__()
        self.model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2').to(device).train()
        self.cos = torch.nn.CosineSimilarity(dim=1, eps=1e-4)
        
    def mean_pooling(self, token_embeddings, attention_mask): 
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def forward(self, input_ids, attention_mask): 
        
        input_ids_a = input_ids[0].to(device)
        input_ids_b = input_ids[1].to(device)
        attention_a = attention_mask[0].to(device)
        attention_b = attention_mask[1].to(device)
        
        #encode sentence and get mean pooled sentence representation 
        encoding1 = self.model(input_ids_a, attention_mask=attention_a)[0] #all token embeddings
        encoding2 = self.model(input_ids_b, attention_mask=attention_b)[0]
        
        print(encoding1.size())
        meanPooled1 = self.mean_pooling(encoding1, attention_a)
        meanPooled2 = self.mean_pooling(encoding2, attention_b)
        
        pred = self.cos(meanPooled1, meanPooled2)
        return pred


In [8]:
first = [1, 2, 3]
second = [4, 5, 6]
first += second
print(first)

[1, 2, 3, 4, 5, 6]


In [9]:
def validateBi(model, validLoader, loss_func):
    model.eval()
   
    preds = []
    gts = []
    
    for batch in tqdm(validLoader): 
        
        input_ids = [batch["text1Merged_input_ids"], batch["text2Merged_input_ids"]]
        attention_masks = [batch["text1Merged_attention_mask"], batch["text2Merged_attention_mask"]]
        pred = model(input_ids, attention_masks)
        gt = batch["ground_truth"].to(device)
        
        preds += list(pred.detach().cpu().tolist())
        gts += list(gt.detach().cpu().tolist())
    corr = np.corrcoef(preds, gts)[1,0]
    print(corr)
    model.train()
    return corr

In [10]:
def trainBi(trainDataset, validDataset): 
    model = BiModel().to(device)
    
    # we would initialize everything first
    optim = torch.optim.Adam(model.parameters(), lr=2e-6)
    
    # and setup a warmup for the first ~10% steps
    total_steps = int(len(trainDataset) / BATCH_SIZE)*EPOCHS
    warmup_steps = int(0.1 * total_steps)
    #TODO: change warmup steps back after 
    scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=warmup_steps, num_training_steps=total_steps - warmup_steps)

    loss_func = torch.nn.MSELoss(reduction="mean")

    trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=BATCH_SIZE, shuffle=True)
    validLoader = torch.utils.data.DataLoader(validDataset, batch_size=2, shuffle=False)
    
    for epoch in range(EPOCHS):
        print("EPOCH: " + str(epoch))
        validateBi(model, validLoader, loss_func)
        
        model.train()  # make sure model is in training mode

        #DEBUGGING
        #prevParam = list(model.parameters())[0].clone()
        for batch in tqdm(trainLoader):
            optim.zero_grad()
            
            input_ids = [batch["text1Merged_input_ids"], batch["text2Merged_input_ids"]]
            attention_masks = [batch["text1Merged_attention_mask"], batch["text2Merged_attention_mask"]]
            pred = model(input_ids, attention_masks)
            
            gt = batch["ground_truth"].to(device)
            loss = loss_func(pred, gt)
            
            # using loss, calculate gradients and then optimize
            loss.backward()
            optim.step()
            scheduler.step()
            
            
            #DEBUGGING
            """
            print(loss)
            print(scheduler.get_last_lr())
            
            #see if model params have changed
            param = list(model.parameters())[0].clone()
            print(torch.equal(param.data, prevParam.data))
            """
    print("final validation")
    validateBi(model, validLoader, loss_func)
    return 

In [11]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=FOLDS, shuffle=True)

In [12]:
metrics = []
transformers.logging.set_verbosity_error()
biTokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')

crossTokenizer = RobertaTokenizer.from_pretrained('roberta-base')

#NOTE: THIS LINE IS ONLY FOR QUICK TRAINING CHECK 
#leanDf = leanDf.iloc[:100, :]

#we only want to sample validation data from the pairs that are both english 
enDf = leanDf[(leanDf["url1_lang"] == "en") & (leanDf["url2_lang"] == "en")]

print("Total df len: " +  str(len(leanDf)))
print("English df len: " +  str(len(enDf)))

biCorrs = []
#we create splits based on the position (not the actual index) of rows in enDf
#the idea is to get a split of the english dataset to set aside and then 
#grab everything else in the en + translated dataset to train on 
for i, (train_index, valid_index) in enumerate(kf.split(enDf)): 
    
    #grab the rows in enDf corresponding to the positions of our split 
    validDf = enDf.iloc[valid_index]
    
    #now get the actual indicies that have been selected
    #and subtract the indices in trainDf away from those 
    remainingIndices = list(set(leanDf.index) - set(validDf.index))
    trainDf = leanDf.loc[remainingIndices]
    validDf = validDf.reset_index(drop=True)
    
    print("###### " + str(i).upper() + " ######")
    print("Train df len: " + str(len(trainDf)))
    print("Valid df len: " + str(len(validDf)))
    
    
    trainDataset = Dataset.from_pandas(trainDf)
    validDataset = Dataset.from_pandas(validDf)
    
    all_cols = ["ground_truth"]
    #NOTE: here we use the merged text
    for part in ["text1Merged", "text2Merged"]: 
        #tokenizes each row of the dataset and gives us back tuple of lists 
        trainDataset = trainDataset.map(lambda x: biTokenizer(x[part], max_length=384, padding="max_length", truncation=True))
        validDataset = validDataset.map(lambda x: biTokenizer(x[part], max_length=384, padding="max_length", truncation=True))
        
        for col in ['input_ids', 'attention_mask']: 
            trainDataset = trainDataset.rename_column(col, part+'_'+col)
            validDataset = validDataset.rename_column(col, part+'_'+col)
            all_cols.append(part+'_'+col)
            
    trainDataset.set_format(type='torch', columns=all_cols)
    validDataset.set_format(type='torch', columns=all_cols)
    
    biCorrs.append(trainBi(trainDataset, validDataset))

    """
    #TRAIN CROSS ENCODER
    #get data loaded in properly 
    trainDataset = Dataset.from_pandas(trainDf)
    validDataset = Dataset.from_pandas(validDf)
    
    #NOTE: here we use the merged text
    trainDataset = trainDataset.map(lambda x: tokenizer(x["text1Merged"], x["text2Merged"], max_length=512, padding="max_length", truncation=True))
    validDataset = validDataset.map(lambda x: tokenizer(x["text1Merged"], x["text2Merged"], max_length=512, padding="max_length", truncation=True))
    

    #only need the input information 
    trainDataset = trainDataset.remove_columns(["text1Merged", "text2Merged", "__index_level_0__"])
    validDataset = validDataset.remove_columns(["text1Merged", "text2Merged", "__index_level_0__"])

    # convert dataset features to PyTorch tensors
    validDataset.set_format(type='torch', columns=["ground_truth", "input_ids", "attention_mask"])
    trainDataset.set_format(type='torch', columns=["ground_truth", "input_ids", "attention_mask"])
    
    
    validMetrics = train(trainDataset, validDataset)
    metrics.append(validMetrics)
    
    del trainDataset
    del validDataset
    
    """
    

Total df len: 4806
English df len: 1738
###### 0 ######
Train df len: 4458
Valid df len: 348


  0%|          | 0/4458 [00:00<?, ?ex/s]

  0%|          | 0/348 [00:00<?, ?ex/s]

  0%|          | 0/4458 [00:00<?, ?ex/s]

  0%|          | 0/348 [00:00<?, ?ex/s]

EPOCH: 0


  0%|          | 0/174 [00:00<?, ?it/s]

torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])
torch.Size([2, 384, 768])


KeyboardInterrupt: 

In [ ]:
#model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = BiModel()

In [ ]:
[param for param in model.parameters()][0][0]

In [ ]:
validDataset = Dataset.from_pandas(validDf)
validDataset[0]

for part in ["text1Merged", "text2Merged"]: 
        #tokenizes each row of the dataset and gives us back tuple of lists 
        validDataset = validDataset.map(lambda x: biTokenizer(x[part], max_length=384, padding="max_length", truncation=True))
        
        for col in ['input_ids', 'attention_mask']: 
            validDataset = validDataset.rename_column(col, part+'_'+col)
            all_cols.append(part+'_'+col)

validDataset[0]

In [ ]:
all_cols = ["ground_truth"]
    for part in ["text1Merged", "text2Merged"]: 
        #tokenizes each row of the dataset and gives us back tuple of lists 
        validDataset = validDataset.map(lambda x: biTokenizer(x[part], max_length=384, padding="max_length", truncation=True))
        
        for col in ['input_ids', 'attention_mask']: 
            validDataset = validDataset.rename_column(col, part+'_'+col)
            all_cols.append(part+'_'+col)
    validDataset.set_format(type='torch', column=all_cols)

In [ ]:
trainLoader = torch.utils.data.DataLoader(validDataset, batch_size=4, shuffle=True)
validDataset[0]

In [ ]:
#send corrList to pickled object for later analysis 
import pickle
out_path = "/home/blitt/projects/localNews/models/sentEmbeddings/2.0-biCrossModel/1.0-justCross/crossCorrList.pckl"
f = open(out_path, "wb")

pickle.dump(biCorrs, f)
